# Partners Network

In [ ]:
import pandas as pd
import networkx as nx
from collections import deque
import numpy as np
import itertools
import matplotlib.pyplot as plt

### Pre-processing


In [ ]:
# Reading the file
df = pd.read_csv(r"T:\Dados\Analytics\0. Data Hub\8. Banco de Dados\Network Socios\Receita Federal - Número de  Empresas por CNAE e UF - v2.csv", sep=',', encoding='latin1', dtype=str)

# Grouping the states (UFs) where each company has establishments
ufs = df.groupby('cnpj_basico')['sigla_uf'].unique().reset_index()
ufs['sigla_uf'] = ufs['sigla_uf'].apply(tuple)

# Grouping the municipalities where each company has establishments
df['nome_municÃ­pio'] = df['nome_municÃ­pio'].str.replace('\\r', '')
municipalities = df.groupby('cnpj_basico')['nome_municÃ­pio'].unique().reset_index()
municipalities['nome_municÃ­pio'] = municipalities['nome_municÃ­pio'].apply(tuple)

# Grouping the primary fiscal CNAEs of each company's establishments
primary_cnaes = df.groupby('cnpj_basico')['cnae_fiscal_primaria'].unique().reset_index()
primary_cnaes['cnae_fiscal_primaria'] = primary_cnaes['cnae_fiscal_primaria'].apply(tuple)

# Grouping the secondary fiscal CNAEs of each company's establishments
def string_to_list(s):
    if type(s) != str:
        return []
    return s.split(',')

df['cnae_fiscal_secundaria'] = df['cnae_fiscal_secundaria'].apply(string_to_list)
df['cnae_fiscal_secundaria'] = df['cnae_fiscal_secundaria'].apply(tuple)

secondary_cnaes = df.groupby('cnpj_basico')['cnae_fiscal_secundaria'].unique().reset_index()
secondary_cnaes['cnae_fiscal_secundaria'] = df['cnae_fiscal_secundaria'].apply(lambda x: tuple(set([val for val in x])))

# Grouping the taxation forms of each company's establishments
taxation_forms = df.groupby('cnpj_basico')['forma_tributacao'].unique().reset_index()
taxation_forms['forma_tributacao'] = taxation_forms['forma_tributacao'].apply(tuple)


In [ ]:
# Merge
df = df[['cnpj_basico', 'company_name', 'social_capital', 'partner_name', 'partner_cnpj_cpf']]

df_ = pd.merge(df, ufs, on='cnpj_basico', how='left')
df_ = pd.merge(df_, municipalities, on='cnpj_basico', how='left')
df_ = pd.merge(df_, primary_cnaes, on='cnpj_basico', how='left')
df_ = pd.merge(df_, secondary_cnaes, on='cnpj_basico', how='left')
df_ = pd.merge(df_, taxation_forms, on='cnpj_basico', how='left')

# Partner identifier
df_['PARTNER_ID'] = df_['partner_name'] + df_['partner_cnpj_cpf']
df_.drop(['partner_name', 'partner_cnpj_cpf'], axis=1, inplace=True)

partners = df_.groupby('cnpj_basico')['PARTNER_ID'].unique().reset_index().rename(columns={'PARTNER_ID': 'PARTNERS'})
partners['PARTNERS'] = partners['PARTNERS'].apply(tuple)

df_ = pd.merge(df_, partners, how='left', on='cnpj_basico')

# Removing duplicates
df_ = df_.drop_duplicates()
df_.rename(columns={'nome_municÃ­pio': 'municipalities'}, inplace=True)

# Adding the number of companies each partner is involved with
occurrences = df_['PARTNER_ID'].value_counts()
df_['NUM_PARTNERSHIPS'] = df_['PARTNER_ID'].map(occurrences)

# Removing partners that are associated with only one company
df_ = df_[df_['NUM_PARTNERSHIPS'] > 1].reset_index().drop('index', axis=1)


In [ ]:
df_

### Product 1
- Edges (common partners)
- Vertices (companies)### Produto 1

In [ ]:
def generate_vertices1(root, data, limit=None):

    def extract_information(data, row, distance):
        # Extract information from the row
        current_cnpj = row['cnpj_basico']
        company_name = row['razao_social_nome_emp']
        partners_count = row['SOCIOS']
        social_capital = row['capital_social']
        states = row['sigla_uf']
        municipalities = row['municipios']
        primary_cnaes = row['cnae_fiscal_primaria']
        secondary_cnaes = row['cnae_fiscal_secundaria']
        partners = data.loc[data['cnpj_basico'] == current_cnpj, 'ID_SOCIO'].tolist()

        return (current_cnpj, company_name, social_capital, states, municipalities, primary_cnaes, secondary_cnaes, distance, partners_count, partners)
    
    # Performs a breadth-first traversal starting from a cnpj_basico (the root of the graph)
    visited = set()
    queue = deque([(root, 0)])
    vertices = []

    while queue:
        current_cnpj, distance = queue.popleft()

        # Do not visit the same vertex twice
        if current_cnpj in visited or (limit is not None and distance > limit):
            continue

        # Extract information of the current company
        current_company = data[data['cnpj_basico'] == current_cnpj].iloc[0]
        vertices.append(extract_information(data, current_company, distance))

        visited.add(current_cnpj)

        # Find all companies connected by partners
        connected_companies = data[data['ID_SOCIO'].isin(vertices[-1][-1])]['cnpj_basico'].unique()

        # Add the connected companies to the queue with the incremented distance
        for company in connected_companies:
            if company not in visited:
                queue.append((company, distance + 1))

    return pd.DataFrame(vertices, columns=['COMPANY', 'COMPANY_NAME', 'SOCIAL_CAPITAL', 'STATES', 'MUNICIPALITIES', 'PRIMARY_CNAEs', 'SECONDARY_CNAEs', 'LEVEL', 'TOTAL_PARTNERS', 'CONNECTED_PARTNERS']).sort_values('LEVEL')


In [ ]:
def generate_edges(data):
    edges = []
    cnpjs = {row['cnpj_basico']: (row[-2], row[3], row[5], row[6]) for index, row in data.iterrows()}
    
    for _, group in data.groupby('ID_SOCIO'):
        companies = group['cnpj_basico'].tolist()

        if len(companies) > 1:
            for name1, name2 in itertools.combinations(companies, 2):
                common_partners = tuple(set(cnpjs[name1][0]).intersection(set(cnpjs[name2][0])))
                num_common_partners = len(common_partners)
                total_partners_union = len(set(cnpjs[name1][0]).union(set(cnpjs[name2][0])))
                partners_percentage = num_common_partners / total_partners_union

                uf_flag = 0 if len(set(cnpjs[name1][1]).intersection(set(cnpjs[name2][1]))) == 0 else 1

                common_primary_cnaes = tuple(set(cnpjs[name1][2]).intersection(set(cnpjs[name2][2])))
                num_primary_cnaes = len(common_primary_cnaes)
                total_primary_cnaes = len(set(cnpjs[name1][2]).union(set(cnpjs[name2][2])))
                primary_cnaes_percentage = num_primary_cnaes / total_primary_cnaes

                common_secondary_cnaes = tuple(set(cnpjs[name1][3]).intersection(set(cnpjs[name2][3])))
                num_secondary_cnaes = len(common_secondary_cnaes)
                total_secondary_cnaes = len(set(cnpjs[name1][3]).union(set(cnpjs[name2][3])))
                if total_secondary_cnaes != 0:
                    secondary_cnaes_percentage = num_secondary_cnaes / total_secondary_cnaes
                else:
                    secondary_cnaes_percentage = None

                edges.append((name1, name2, common_partners, num_common_partners, partners_percentage, uf_flag, common_primary_cnaes, primary_cnaes_percentage, common_secondary_cnaes, num_secondary_cnaes, secondary_cnaes_percentage))

    return pd.DataFrame(edges, columns = ['COMPANY_1', 'COMPANY_2', 'COMMON_PARTNERS', 'NUM_COMMON_PARTNERS', 'PERCENTAGE_COMMON_PARTNERS', 'UF_FLAG', 'COMMON_PRIMARY_CNAES', 'PERCENTAGE_PRIMARY_CNAES',
                                          'COMMON_SECONDARY_CNAES', 'NUM_COMMON_SECONDARY_CNAES', 'PERCENTAGE_SECONDARY_CNAES'])


In [ ]:
def search_companies_connected_by_partners_V2(cnpj_b: str, data, distance_limit=None):
    
    # Vertices
    df_vertices = generate_vertices1(cnpj_b, data, distance_limit)
    df_vertices['PARENTS'] = ""

    # Edges
    df_edges = generate_edges(data[data['cnpj_basico'].isin(df_vertices['COMPANY'])])

    # Adding reference to the parent node(s)
    level_dict = df_vertices.set_index('COMPANY')['LEVEL'].to_dict()

    for ind, edge in df_edges.iterrows():
        if level_dict[edge['COMPANY_1']] - level_dict[edge['COMPANY_2']] == 1:
            df_vertices.loc[df_vertices['COMPANY'] == edge['COMPANY_1'], 'PARENTS'] += ("," + edge['COMPANY_2'])
        if level_dict[edge['COMPANY_2']] - level_dict[edge['COMPANY_1']] == 1:
            df_vertices.loc[df_vertices['COMPANY'] == edge['COMPANY_2'], 'PARENTS'] += ("," + edge['COMPANY_1'])

    return df_vertices, df_edges


In [ ]:
vertices, edges = search_companies_connected_by_partners_V2('13930213', df_, 4)
vertices


### Product 2
- Creation of the graph using networkx
- Adjacency Matrix
- Traversal

In [ ]:
def generate_graph(v, a):
    
    g = nx.Graph()

    # Adding vertices to the graph
    for ind, vertex in v.iterrows():
        g.add_node(vertex[0], company_name=vertex[1], social_capital=vertex[2], states=vertex[3], municipalities=vertex[4],
                   primary_cnaes=vertex[5], secondary_cnaes=vertex[6], level=vertex[7], total_partners=vertex[8], connected_partners=vertex[9])

    # Adding edges to the graph
    for ind, edge in a.iterrows():
        g.add_edge(edge[0], edge[1], common_partners=edge[2], common_primary_cnaes=edge[6], common_secondary_cnaes=edge[8], num_common_partners=edge[3], partners_percentage=edge[4], 
                   uf_weight=edge[5], primary_cnaes_weight=edge[7], secondary_cnaes_weight=edge[9], secondary_cnaes_percentage=edge[10])

    return g


In [ ]:
g = generate_graph(vertices, edges)

# Adjacency Matrix
nx.to_pandas_adjacency(g, weight='secondary_cnaes_percentage')

In [ ]:
def find_paths(g, source, target, cut_off=7, weight='num_common_partners'):

    # Check if the source and target nodes are in the graph
    if source not in g.nodes() or target not in g.nodes():
        raise ValueError("The source and target nodes MUST be in the graph.")

    # Find all simple paths between the source and target
    paths = list(nx.all_simple_paths(g, source=source, target=target, cutoff=cut_off))

    # Return
    res = []
    for path in paths:
        path_length = len(path) - 1  # The path length is the number of edges
        company_names = [g.nodes[node]['company_name'] for node in path]
        total_weight = sum(g[path[i]][path[i + 1]][weight] for i in range(path_length))
        common_partners = list(g[path[i]][path[i + 1]]['common_partners'] for i in range(path_length))
        
        res.append((path, company_names, common_partners, path_length, total_weight))
    
    def sorting_key(tup):
        return (tup[-2], -tup[-1])
    
    return sorted(res, key=sorting_key)


In [ ]:
paths = find_paths(g, '13930213', '37228135')


In [ ]:
def plot_top5(paths):

    start = paths[0][0][0]
    end = paths[0][0][-1]

    best_path = paths[0][0]
    edges = [(best_path[i], best_path[i + 1]) for i in range(len(best_path) - 1)]

    subV = []
    for p in paths[:20]:
        subV += p[0]
    
    subV = list(set(subV))
    subG = g.subgraph(subV)

    pos = nx.spring_layout(subG)

    plt.figure(figsize=(20, 10))

    node_colors = ['red' if v == start else 'blue' if v == end else 'gray' for v in subG.nodes()]
    edge_colors = ['red' if (edge[0], edge[1]) in edges or (edge[1], edge[0]) in edges else 'gray' for edge in subG.edges()]
    edge_widths = [subG[edge[0]][edge[1]]['num_common_partners'] if 'num_common_partners' in subG[edge[0]][edge[1]] else 1.0 for edge in subG.edges()]

    nx.draw(subG, pos, with_labels=True, font_weight='bold', node_color=node_colors, edge_color=edge_colors, width=edge_widths)

    edge_labels = nx.get_edge_attributes(subG, 'num_common_partners')
    nx.draw_networkx_edge_labels(subG, pos, edge_labels=edge_labels)

    plt.show()

In [ ]:
plot_top5(paths)

## RANKING

In [ ]:
paths

In [ ]:
def rank_paths_by_partners(g, source, max_path_length=None):

    res = []

    for node in g.nodes():

        if node != source:
            try:
                res.append((find_paths(g, source, node, cut_off=max_path_length)[0], source, node))
            except:
                res.append(([float('inf'), float('inf')], source, node))
    
    def sorting_key(tup):
        return (tup[0][-2], -tup[0][-1])
    
    return pd.DataFrame(sorted(res, key=sorting_key))


In [ ]:
test = rank_paths_by_partners(g, '13930213', max_path_length=5)
test